In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise02AI import Exercise02AI, display_path

# Aufgabe 03: Minimax (Simplified Evaluation Function)

Diese Klasse implementiert die Simplified Evaluation Function zur Berechnung des nächsten Zuges im Mittelspiel. Hierbei wird die `minimax`-Funktion der Super-Klasse (`Exercise02AI`) aufgerufen und durch die nun implementierte Simplified Evaluation Function verbessert.

Hierzu werden zu Beginn die in der [Simplified Evaluation Function](https://www.chessprogramming.org/Simplified_Evaluation_Function#Piece-Square_Tables) beschriebenen `Piece-Square Tables` definiert. Diese weisen jeder der 64 möglichen Positionen auf dem Schachfeld für alle der Schachfiguren (Bauer, Springer, Läufer, Turm, Dame und König) eine numerische Bewertung in Form einer Ganzzahl zu. Diese kann sowohl positiv (gute Position für die Figur) oder negativ (schlechte Position für die Figur) sein und wird als Liste dargestellt. Theoretisch verfügt der König über zwei verschiedene Listen, eine für das Mittelspiel, eine für das Endspiel. Da das Endspiel aber von der Syzygy-Bibliothek übernommen wird, fällt diese weg und es reicht die Implementierung der List für das Mittelspiel.

Anhand der Piece-Square Table der Bauern `PST_PAWN` die im Folgenden definiert ist, wird ersichtlich, dass Bauern auf den Felder `f2`, `g2` und `h2` (symmetrischer Weise natürlich auch auf `a2`, `b2` und `c2`) Boni (positive Bewertung) erhalten, während sie auf den Felder `f3` und `g3` Abzüge (negative Bewertungen) erhalten.

Das Dictionary `PIECE_TO_PST` weist jedem Figurentyp die entsprechende umgedrehte Piece-Square Table zu, welche aufgrund der einfacheren Lesbarkeit in umgekehrter Reihenfolge initialisiert werden.

In [ ]:
import chess

class Exercise03AI(Exercise02AI):
    """Chooses middle game moves using minimax algorithm and piece square tables"""
    def __init__(self, player_name: str, search_depth: int = 2) -> None:
        super().__init__(player_name, search_depth)
        # Create pice square table mapping (needs to revert tables) 
        self.PIECE_TO_PST = {
            chess.PAWN:   self.PST_PAWN[::-1],
            chess.KNIGHT: self.PST_KNIGHT[::-1],
            chess.BISHOP: self.PST_BISHOP[::-1],
            chess.ROOK:   self.PST_ROOK[::-1],
            chess.QUEEN:  self.PST_QUEEN[::-1],
            chess.KING:   self.PST_KING[::-1]
        }

    # Define field values as class constants
    PST_PAWN = [
        0,  0,  0,  0,  0,  0,  0,  0,
       50, 50, 50, 50, 50, 50, 50, 50,
       10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 25, 25, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ]

    PST_KNIGHT = [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50
    ]

    PST_BISHOP = [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20
    ]

    PST_ROOK = [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
        0,  0,  0,  5,  5,  0,  0,  0
    ]

    PST_QUEEN = [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
         -5,  0,  5,  5,  5,  5,  0, -5,
          0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ]

    PST_KING = [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
         20, 20,  0,  0,  0,  0, 20, 20,
         20, 30, 10,  0,  0, 10, 30, 20
    ]

## Evaluierungsfunktion

Die Funktion `calculate_pst_value` nimmt als Argument ein Board, einen Figurentyp und eine Farbe. Anhand der gegebenen Farbe wird auf dem Board nach den Figuren des gegebenen Typs gefiltert, deren Position auf dem Board bestimmt (für schwarze Figuren wird das Board gespiegelt) und mithilfe der Piece-Square Tables eine Bewertung dieser vollzogen. Diese Bewertungen werden, falls schwarz als Farbe gegeben ist, negiert. Die Bewertungen werden im letzten Schritt aufsummiert und zurückgegeben.

In [ ]:
class Exercise03AI(Exercise03AI): # type: ignore
    def calculate_pst_value(self, board: chess.Board, piece: chess.PieceType, color: chess.Color):
        """Calculates the piece-square table values from given color"""  
        if color == chess.WHITE:
            return sum([self.PIECE_TO_PST[piece][i] for i in board.pieces(piece, color)])
        else:
            return sum([-(self.PIECE_TO_PST[piece][chess.square_mirror(i)]) for i in board.pieces(piece, color)])

Die Funktion `evaluate` berechnet den Wert eines gesamten Boards, welches als Argument übergeben wird. Als Grundlage werden die Materialwerte der einzelnen Figuren in Centipawns benötigt und wie folgt gesetzt um in der Elternklasse überschrieben zu werden:

| Figurname  | Materialwert |
|---|---|
| Bauer (pawn) | 100  |
| Springer (knight) | 320  |
| Läufer (bishop) | 330  |
| Turm (rook) | 500  |
| Dame (queen) | 900  |

Die Funktion `evaluate` berechnet den Wert des Boards wie folgt:

1. Berechnung aller weißen und schwarzen Figuren mithilfe der Funktion `calculate_pst_value`
2. Erstellung der Liste `rating` die die Bewertung aller Figuren des Boards enthält
3. Speicherung des Aufrufs von `evaluate` aus der Elterklasse in der Variable `material` um ein Maß für die Anzahl der Figuren im Verhältnis von weiß zu schwarz zu erhalten
4. Rückgabe der Summe von `rating` und `material`

In [ ]:
class Exercise03AI(Exercise03AI): # type: ignore
    MATERIAL_VALUE_PAWN = 100
    MATERIAL_VALUE_KNIGHT = 320
    MATERIAL_VALUE_BISHOP = 330
    MATERIAL_VALUE_ROOK = 500
    MATERIAL_VALUE_QUEEN = 900

    def evaluate(self, board: chess.Board) -> int:
        """Evaluates a given board. 
        Returns a positive value if white has a better material value than black."""
        # Get if pieces standing well or pieces standing badly by calculating with the Piece-Square Tables
        rating_pawn_white   = self.calculate_pst_value(board, chess.PAWN,   chess.WHITE)
        rating_pawn_black   = self.calculate_pst_value(board, chess.PAWN,   chess.BLACK)
        rating_knight_white = self.calculate_pst_value(board, chess.KNIGHT, chess.WHITE)
        rating_knight_black = self.calculate_pst_value(board, chess.KNIGHT, chess.BLACK)
        rating_bishop_white = self.calculate_pst_value(board, chess.BISHOP, chess.WHITE)
        rating_bishop_black = self.calculate_pst_value(board, chess.BISHOP, chess.BLACK)
        rating_rook_white   = self.calculate_pst_value(board, chess.ROOK,   chess.WHITE)
        rating_rook_black   = self.calculate_pst_value(board, chess.ROOK,   chess.BLACK)
        rating_queen_white  = self.calculate_pst_value(board, chess.QUEEN,  chess.WHITE)
        rating_queen_black  = self.calculate_pst_value(board, chess.QUEEN,  chess.BLACK)
        rating_king_white   = self.calculate_pst_value(board, chess.KING,   chess.WHITE)
        rating_king_black   = self.calculate_pst_value(board, chess.KING,   chess.BLACK)

        rating = [
            rating_pawn_white,   rating_pawn_black, 
            rating_knight_white, rating_knight_black, 
            rating_bishop_white, rating_bishop_black, 
            rating_rook_white,   rating_rook_black, 
            rating_queen_white,  rating_queen_black, 
            rating_king_white,   rating_king_black
        ]
        # Get material value
        material = super().evaluate(board)
        return sum(rating) + material

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
board = chess.Board()
board.set_fen("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise03AI("Testplayer", 3)
print(player.evaluate(board))
board

In [ ]:
DEPTH = 3
board = chess.Board()
board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise03AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.best_move(board)
print(move)
display_path(board, move, DEPTH)